In [1]:
import sys
import os
import csv
import random
from keras.layers import *
from keras.optimizers import *
from keras.applications import *
from keras.models import Model
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras import backend as k
from keras.preprocessing.image import load_img, img_to_array
from keras import backend as K
import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
from force_utils import DataProcessing

Using TensorFlow backend.


In [16]:
# -------------------------------------------------------------------------------------
# DATASET
# -------------------------------------------------------------------------------------

# Input
x_train = np.load('./dataset/features/vgg16_train.npy')
x_test = np.load('./dataset/features/vgg16_val.npy')


# Target
video_file = 'dataset/video.mp4'
csv_file = 'dataset/forceinfo.csv'
dataset = DataProcessing(video_file, csv_file, p=True, f=True, name=False)

# Split train and validation
keys = list(dataset.p_dict.keys())
values = list(dataset.p_dict.values())
train_num = int(round(0.8 * len(keys)))

# Adjust to Input shape
y_train = values[:train_num]
for x in values[:30]:
    y_train.append(x)
y_train = np.array(y_train)

y_test = values[train_num:]
for x in values[train_num:][:8]:
    y_test.append(x)
y_test = np.array(y_test)

In [19]:
# -------------------------------------------------------------------------------------
# VARIABLE
# -------------------------------------------------------------------------------------

input_shape = (299, 299, 3)
batch_size = 32
epochs = 30
learning_rate = 0.01

In [20]:
# -------------------------------------------------------------------------------------
# MODEL
# -------------------------------------------------------------------------------------

input_tensor = Input(shape=x_train.shape[1:])
x = Flatten(input_shape=x_train.shape[1:])(input_tensor)
x = Dense(512, activation='relu')(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
prediction = Dense(1)(x)
model = Model(inputs=input_tensor, outputs=prediction)

In [21]:
# -------------------------------------------------------------------------------------
# COMPILE
# -------------------------------------------------------------------------------------

# Optimizer
optim = RMSprop(lr=0.001, rho=0.9, epsilon=None, decay=0.0)

# Learning rate scheduler
def schedule(epoch, decay=0.9):
    return learning_rate * decay**(epoch)

# Callbacks
callbacks = [
    ModelCheckpoint('dataset/weights/p_npy_mae_{epoch:02d}.h5', monitor='val_loss', verbose=1, save_best_only=True),
    EarlyStopping(monitor='val_loss', min_delta=0, patience=5, verbose=1, mode='auto')]
    # LearningRateScheduler(schedule)]

# Compile
model.compile(optimizer=optim, loss='mean_absolute_error')

In [22]:
# -------------------------------------------------------------------------------------
# FIT
# -------------------------------------------------------------------------------------

history = model.fit(
    x_train, y_train,
    batch_size=batch_size,
    epochs=epochs,
    callbacks=callbacks,
    validation_data=(x_test, y_test)
)

Train on 45312 samples, validate on 11328 samples
Epoch 1/30
45312/45312 [==============================] - 20s 444us/step - loss: 0.3327 - val_loss: 0.1023

Epoch 00001: val_loss improved from inf to 0.10231, saving model to dataset/weights/p_npy_mae_01.h5
Epoch 2/30
45312/45312 [==============================] - 19s 412us/step - loss: 0.1590 - val_loss: 0.1024

Epoch 00002: val_loss did not improve from 0.10231
Epoch 3/30
45312/45312 [==============================] - 19s 414us/step - loss: 0.1582 - val_loss: 0.1039

Epoch 00003: val_loss did not improve from 0.10231
Epoch 4/30
45312/45312 [==============================] - 19s 413us/step - loss: 0.1578 - val_loss: 0.1020

Epoch 00004: val_loss improved from 0.10231 to 0.10198, saving model to dataset/weights/p_npy_mae_04.h5
Epoch 5/30
45312/45312 [==============================] - 19s 412us/step - loss: 0.1576 - val_loss: 0.1028

Epoch 00005: val_loss did not improve from 0.10198
Epoch 6/30
45312/45312 [=============================